# Evaluation Malayalam Speech Corpus(MSC) dataset

In [ ]:
#| default_exp msc

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export
import time
from typing import List

import pandas as pd
from datasets import load_dataset, Audio
from jiwer import wer, cer
from transformers import pipeline

from malayalam_asr_benchmarking.utils import whisper_norm, is_target_text_in_range, get_text, normalise, data, get_model_size, clear_gpu_memory

## Loading dataset and evaluating model

In [ ]:
#| export
def load_malayalam_speech_corpus_dataset():
    dataset = load_dataset(
            "thennal/msc",
            split="train"
    )
    dataset = dataset.cast_column("audio", Audio(sampling_rate=16000))
    dataset = dataset.map(normalise)
    dataset = dataset.filter(is_target_text_in_range, input_columns=["norm_text"])
    return dataset

In [ ]:
#| export
def evaluate_whisper_model_msc(
        model_name: str, # The model name
        werlist: List[float], # WER List
        cerlist: List[float], # CER list
        modelsizelist: List[str], # model size list
        timelist: List[float], # time(s) list
        bs:int =16, # batch size
)->None:
    whisper_asr = pipeline(
            "automatic-speech-recognition", model=model_name, device=0
        )
    dataset = load_malayalam_speech_corpus_dataset()
    
    predictions = []
    references = []

    start = time.time()
    for out in whisper_asr(data(dataset), batch_size=bs):
        predictions.append(whisper_norm(out["text"]))
        references.append(out["reference"][0])
        
        
    end = time.time()
    print(f"Total time taken: {end - start}")
    timelist.append(end - start)
    
    df = pd.DataFrame({"predictions": predictions, "ground_truth": references})
    df["model_name"] = model_name
    df["wer"] = df.apply(lambda row: wer(row["ground_truth"], row["predictions"]), axis=1)
    df["cer"] = df.apply(lambda row: cer(row["ground_truth"], row["predictions"]), axis=1)
    df["total_time"] = end-start
    
    rwer = wer(references, predictions)
    rwer = round(100 * rwer, 2)
    df["total_wer"] = rwer
    werlist.append(rwer)
    print(f"The WER of model: {rwer}")

    rcer = cer(references, predictions)
    rcer = round(100 * rcer, 2)
    df["total_cer"] = rcer
    cerlist.append(rcer)
    print(f"The CER of model: {rcer}")
    
    print(f"The model size is: {get_model_size(whisper_asr.model)}")
    modelsizelist.append(get_model_size(whisper_asr.model))
    df["model_size"] = get_model_size(whisper_asr.model)
    
    save_name = model_name.split("/")
    print(save_name)
    df.to_parquet(f"/home/msc_results/{save_name[0]}_{save_name[1]}_msc.parquet")
    
    clear_gpu_memory()

## Testing with a sample model

In [ ]:
#|eval: false
wer_list = []
cer_list = []
model_size_list = []
time_list = []
evaluate_whisper_model_msc("parambharat/whisper-tiny-ml", wer_list, cer_list, model_size_list, time_list)

Found cached dataset parquet (/home/.cache/huggingface/datasets/thennal___parquet/thennal--msc-cc9d10989b2ac4bd/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec)
Loading cached processed dataset at /home/.cache/huggingface/datasets/thennal___parquet/thennal--msc-cc9d10989b2ac4bd/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec/cache-30f1618974cdefce.arrow
Loading cached processed dataset at /home/.cache/huggingface/datasets/thennal___parquet/thennal--msc-cc9d10989b2ac4bd/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec/cache-e4f860ca9b159c26.arrow
/opt/conda/lib/python3.8/site-packages/transformers/generation_utils.py:1359: UserWarning: Neither `max_length` nor `max_new_tokens` has been set, `max_length` will default to 448 (`self.config.max_length`). Controlling `max_length` via the config is deprecated and `max_length` will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to contr

Total time taken: 772.3569941520691
The WER of model: 43.95
The CER of model: 25.78
The model size is: 37.76M
['parambharat', 'whisper-tiny-ml']


In [ ]:
#|eval: false
wer_list,cer_list, time_list, model_size_list

([43.95], [25.78], [772.3569941520691], ['37.76M'])